In [2]:
# Install Tesseract OCR
!apt-get update
!apt-get install -y tesseract-ocr
# Install pytesseract and Pillow for Python
!pip install pytesseract Pillow jiwer transformers


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Pack

In [32]:
import pytesseract
from PIL import Image
from jiwer import wer
from transformers import MarianMTModel, MarianTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util
import string
import torch
from nltk.translate.bleu_score import sentence_bleu

# Initialize Sentence-BERT model for semantic similarity (multilingual version)
semantic_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

def calculate_bleu(reference, hypothesis):
    reference = [reference.split()]
    hypothesis = hypothesis.split()
    return sentence_bleu(reference, hypothesis)


def calculate_wer(reference, hypothesis):
    return wer(reference, hypothesis)


def normalize_text(text):
    """More robust text normalization"""
    text = text.lower().strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join(text.split())
    return text


def ocr_extraction(image_path):
    try:
        img = Image.open(image_path)
        return pytesseract.image_to_string(img)
    except Exception as e:
        print(f"OCR Error: {str(e)}")
        return ""


def load_translation_model():
    model_name = "Helsinki-NLP/opus-mt-en-fr"
    return (
        MarianMTModel.from_pretrained(model_name),
        MarianTokenizer.from_pretrained(model_name)
    )

def translate_text(model, tokenizer, text):
    try:
        inputs = tokenizer([text], return_tensors="pt", truncation=True, max_length=512)
        translated = model.generate(**inputs)
        return tokenizer.decode(translated[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Translation Error: {str(e)}")
        return ""

# Semantic similarity calculation using Sentence-BERT
def calculate_semantic_similarity(ref, hyp):
    """Calculate cosine similarity between reference and hypothesis"""
    if not ref or not hyp:
        return 0.0

    # Encode both sentences
    embeddings = semantic_model.encode([ref, hyp], convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_score = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    return cosine_score.item()


def evaluate_ocr(ocr_output, ground_truth_ocr):
    clean_ocr = normalize_text(ocr_output)
    clean_gt = normalize_text(ground_truth_ocr)
    return wer(clean_gt, clean_ocr)


def evaluate_translation(ocr_text, ground_truth_fr, model, tokenizer):
    """Evaluate translation using semantic similarity"""
    # Translate OCR text
    translated = translate_text(model, tokenizer, normalize_text(ocr_text))

    # Clean both texts
    clean_translation = normalize_text(translated)
    clean_ground_truth = normalize_text(ground_truth_fr)

    # Calculate scores
    similarity = calculate_semantic_similarity(clean_ground_truth, clean_translation)
    bleu_score = calculate_bleu(clean_ground_truth, clean_translation)

    return similarity, bleu_score, translated  # Return both scores and translation

def process_image_for_evaluation(image_path, ground_truth_ocr, ground_truth_fr):
    """Complete processing pipeline"""
    # OCR Extraction
    ocr_text = ocr_extraction(image_path)
    print(f"Extracted OCR Text:\n{ocr_text}\n")

    # OCR Evaluation
    wer_score = evaluate_ocr(ocr_text, ground_truth_ocr)
    print(f"OCR WER: {wer_score:.4f}")

    # Load translation model
    model, tokenizer = load_translation_model()

    # Translation Evaluation
    similarity, bleu, translated_text = evaluate_translation(ocr_text, ground_truth_fr, model, tokenizer)

    # Print translation and scores
    print(f"\nGenerated Translation:\n{translated_text}\n")
    print(f"Semantic Similarity Score: {similarity:.4f}")
    print(f"BLEU Score: {bleu:.4f}")

if __name__ == "__main__":
    image_path = "ocr2.png"  # Update with your image path

    ground_truth_ocr = "2.3 Deep3DFace Deep3DFace is the most advanced model in this comparison, using a deep neural network to provide highly detailed 3D face reconstructions, including accurate facial expressions and fine details like skin texture. This model excels in producing realistic faces and is suited for tasks that require high accuracy. Strengths: • High Detail and Realism: Deep3DFace captures fine details of the face. • Expression Transfer: Deep3DFace excels at transferring facial expressions from one face to another with precision, making it perfect for applications like video game avatars or virtual reality. Limitations: • Processing Time: Deep3DFace takes the longest to process among the three models due to the high level of detail it captures, making it less suitable for real-time applications. • High Computational Requirements: This model requires a powerful system with advanced GPU capabilities to run efficiently, particularly when processing large datasets. Example Use: Deep3DFace is ideal for applications, where realistic and detailed 3D face modeling is required, and processing time is less of a concern. In summary: • If speed and real-time performance are the priority, FaceMesh is the best option. • If more accuracy in facial expressions is required, 3DDFA offers a balanced choice between speed and detail. • For the highest quality and realism in 3D face reconstruction and expression transfer, Deep3DFace is the best model, though it requires more processing power."
    # ground_truth_ocr = "During a shift in the operating room, Radar O'Reilly (Gary Burghoff) enters and informs Henry Blake that Blake has received all of the needed Army service points to be discharged and sent home. Henry begins planning his return and places a telephone call to Bloomington, Illinois, to inform his wife and family of the good news. Majors Margaret Houlihan (Loretta Swit) and Frank Burns (Larry Linville) celebrate privately that Frank will become the unit commander. Henry and Radar share a sentimental moment in which young Radar describes Henry as a father figure, and gives Henry an inscribed Winchester cartridge; a surprised Henry returns the favor by spontaneously giving Radar a rectal thermometer that once belonged to his father. On the night before Henry's departure, Hawkeye Pierce (Alan Alda), Trapper McIntyre (Wayne Rogers), and Radar throw a drunken going-away party for Henry at Rosie's Bar and Grill, and present him with a tailored civilian suit as a parting gift. The next morning, Frank attempts to assemble the company for a formal send-off, but Hawkeye and Trapper are out of uniform and unshaven, and Corporal Klinger (Jamie Farr) wears a particularly elaborate dress made specially for the occasion. Henry arrives in his new suit, and Frank and Margaret give Blake a formal salute, but Henry chides Frank for being too disciplined. Henry's affectionate individual goodbyes to the others are cut short by the imminent arrival of his helicopter, but Hawkeye pulls him aside and persuades him to give a long parting kiss to Margaret, to her surprise and Frank's annoyance."
    # ground_truth_french = "Lors d'une garde au bloc opératoire, Radar O'Reilly (Gary Burghoff) entre et informe Henry Blake que ce dernier a accumulé tous les points de service militaire nécessaires pour être démobilisé et renvoyé chez lui. Henry prépare son retour et téléphone à Bloomington, dans l'Illinois, pour annoncer la bonne nouvelle à sa femme et à sa famille. Les majors Margaret Houlihan (Loretta Swit) et Frank Burns (Larry Linville) célèbrent en privé la nomination de Frank au poste de commandant d'unité. Henry et Radar partagent un moment d'émotion intense : le jeune Radar décrit Henry comme une figure paternelle et lui offre une cartouche Winchester gravée. Henry, surpris, lui rend la pareille en lui offrant spontanément un thermomètre rectal ayant appartenu à son père. La veille du départ d'Henry, Hawkeye Pierce (Alan Alda), Trapper McIntyre (Wayne Rogers) et Radar organisent une fête d'adieu arrosée pour Henry au Rosie's Bar and Grill, et lui offrent un costume civil sur mesure en guise de cadeau d'adieu. Le lendemain matin, Frank tente de réunir la compagnie pour un départ officiel, mais Hawkeye et Trapper sont en civil et mal rasés, et le caporal Klinger (Jamie Farr) porte une robe particulièrement élaborée, confectionnée spécialement pour l'occasion. Henry arrive dans son nouveau costume, et Frank et Margaret saluent Blake de manière solennelle, mais Henry reproche à Frank d'être trop discipliné. Les adieux affectueux d'Henry aux autres sont interrompus par l'arrivée imminente de son hélicoptère, mais Hawkeye le prend à part et le persuade d'embrasser longuement Margaret, à la surprise de celle-ci et au grand dam de Frank."
    ground_truth_french = "2.3 Deep3DFace Deep3DFace est le modèle le plus avancé de cette comparaison. Il utilise un réseau neuronal profond pour fournir des reconstructions faciales 3D très détaillées, incluant des expressions faciales précises et des détails fins comme la texture de la peau. Ce modèle excelle dans la création de visages réalistes et convient aux tâches exigeant une grande précision. Points forts : • Détail et réalisme élevés : Deep3DFace capture les détails les plus fins du visage. • Transfert d'expressions : Deep3DFace excelle dans le transfert précis des expressions faciales d'un visage à l'autre, ce qui le rend idéal pour des applications telles que les avatars de jeux vidéo ou la réalité virtuelle. Contraintes : • Temps de traitement : Deep3DFace est le modèle le plus long à traiter parmi les trois en raison du niveau de détail élevé qu'il capture, ce qui le rend moins adapté aux applications temps réel. • Besoins de calcul élevés : Ce modèle nécessite un système puissant doté de capacités GPU avancées pour fonctionner efficacement, en particulier lors du traitement de grands ensembles de données. Exemple d'utilisation : Deep3DFace est idéal pour les applications nécessitant une modélisation faciale 3D réaliste et détaillée, et où le temps de traitement est moins important. En résumé : • Si la vitesse et les performances en temps réel sont primordiales, FaceMesh est la meilleure option. • Si une plus grande précision des expressions faciales est requise, 3DDFA offre un choix équilibré entre vitesse et détail. • Pour une qualité et un réalisme optimaux lors de la reconstruction faciale 3D et du transfert d'expressions, Deep3DFace est le meilleur modèle, bien qu'il nécessite une puissance de traitement plus importante."

    process_image_for_evaluation(image_path, ground_truth_ocr, ground_truth_french)

Extracted OCR Text:
2.3. Deep3DFace

Deep3DFace is the most advanced model in this comparison, using a deep neural network to provide highly detailed 3D
face reconstructions, including accurate facial expressions and fine details like skin texture. This model excels in producing
realistic faces and is suited for tasks that require high accuracy.

Strengths:

e High Detail and Realism: Deep3DFace captures fine details of the face.

e Expression Transfer: Deep3DFace excels at transferring facial expressions from one face to another with precision,
making it perfect for applications like video game avatars or virtual reality.

Limitations:

e Processing Time: Deep3DFace takes the longest to process among the three models due to the high level of detail it
captures, making it less suitable for real-time applications.

e High Computational Requirements: This model requires a powerful system with advanced GPU capabilities to
run efficiently, particularly when processing large datasets.

Exam

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



Generated Translation:
23 deep3dface deep3dface est le modèle le plus avancé dans cette comparaison en utilisant un réseau neuronal profond pour fournir des reconstructions 3d très détaillées comprenant des expressions faciales précises et des détails fins comme la texture de la peau ce modèle excelle dans la production de visages réalistes et est adapté aux tâches qui nécessitent des forces de haute précision e haute précision et réalisme deep3dface capture des détails fins du transfert de visage e expression deep3dface excelle dans le transfert d'expressions faciales d'un visage à l'autre avec précision ce qui le rend parfait pour des applications comme les avatars de jeu vidéo ou les limitations de réalité virtuelle e traitement deep3dface prend le plus de temps pour traiter parmi les trois modèles en raison du haut niveau de détail qu'il capture rend moins adapté pour des applications en temps réel e hautes exigences informatiques ce modèle nécessite un système puissant avec des c